In [1]:
import os
os.environ["MLFLOW_TRACKING_URI"]="https://github.com/piyushmaps/datascienceproject.git"
os.environ["MLFLOW_TRACKING_USERNAME"]="thakurpiyush1708"
os.environ["MLFLOW_TRACKING_PASSWORD"]="ad8a8e194cf983caeafbe574f27bfe6b693d4e15"
#tracking uri is from clicking remote and the first link
#tracking pass is from dvc and data=> dvc=? secret access key

In [2]:
import os
%pwd

'c:\\Projects\\datascienceproject\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Projects\\datascienceproject'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path  
    metric_file_name: Path
    all_params: dict
    target_column: str
    mlflow_uri: str


In [6]:
from src.datascience.constants import CONFIG_FILE_PATH,PARAMS_FILE_PATH,SCHEMA_FILE_PATH
from src.datascience.utils.common import read_yaml, create_directories, save_json
#save_json help to save our metrics in form of json

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])
    

    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config=self.config.model_evaluation
        params=self.params.ElasticNet
        schema=self.schema.Target_Column

        create_directories([config.root_dir])


        model_evaluation_config=ModelEvaluationConfig(
            root_dir=config.root_dir,
            test_data_path=config.test_data_path,
            model_path=config.model_path,  
            metric_file_name=config.metric_file_name,
            all_params=params,
            target_column=schema.name,
            mlflow_uri="https://github.com/piyushmaps/datascienceproject.git"
        )

        return model_evaluation_config

In [ ]:
import os
import pandas as pd
from sklearn.metrics import mean_squared_error,mean_absolute_error, r2_score
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
import numpy as np
import joblib

class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config=config

    def evaluation_metrics(self,actual, pred):
        rmse=np.sqrt(mean_squared_error(actual,pred))
        mae=np.sqrt(mean_absolute_error(actual,pred))
        r2=r2_score(actual,pred)
        return rmse,mae,r2
    
    def log_into_mlflow(self):
        test_data=pd.read_csv(self.config.test_data_path)
        model=joblib.load(self.config.model_path)

        test_x=test_data.drop([self.config.target_column],axis=1)
        test_y=test_data[[self.config.target_column]]


        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store=urlparse(mlflow.get_tracking_uri()).scheme


        with mlflow.start_run():
            predicted_qualities=model.predict(test_x)

            (rmse,mae,r2)=self.evaluation_metrics(test_y,predicted_qualities)

            #saving metrics as local

            scores={"rmse": rmse,"mae": mae,"r2":r2}

            save_json(path=Path(self.config.metric_file_name),data=scores)

            mlflow.log_params(self.config.all_params)

            mlflow.log_metric("rmse",rmse)
            mlflow.log_metric("mae",mae)
            mlflow.log_metric("rmse",r2)



            if tracking_url_type_store != "file":
                mlflow.sklearn.log_model(model,"model",registered_model_name="ElasticnetModel")
            else:
                mlflow.sklearn.log_model